In [2]:
import mysql.connector as mysql
import plotly.graph_objects as go
import pandas as pd

In [3]:
conn = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "asddsa",
    database = "fids_2"
)
cursor = conn.cursor()

df = pd.read_csv('airports.csv')
df = df.where(df["iso_country"]=="IR")  
df = df.dropna(subset=['id'])
idents = df.loc[:,"ident"] 
idents = idents.unique().tolist()
arrival = {}
departure={}

for id in idents:
    sql = '''SELECT COUNT(*) FROM fids WHERE destination_icao="{}" AND type_="{}"'''.format(id,"departure")
    cursor.execute(sql)
    recs = cursor.fetchall()
    for row in recs:
        departure_n = row[0]
        departure[id]=departure_n

    sql = '''SELECT COUNT(*) FROM fids WHERE destination_icao="{}" AND type_="{}"'''.format(id,"arrival")
    cursor.execute(sql)
    recs = cursor.fetchall()
    for row in recs:
        arrival_n = row[0]
        arrival[id]=arrival_n

df = pd.DataFrame(df)
arrival = arrival.values()
departure=departure.values()
df['arrival'] = arrival
df['departure'] = departure

df.to_csv('out.csv')


In [4]:
df = pd.read_csv('out.csv')
df['text'] = df['name'] + '<br>' + 'Arrivals: ' + df['arrival'].astype(str)+ '<br>' + 'departure: ' + df['departure'].astype(str)

fig = go.Figure(data=go.Scattergeo(
        lon = df['longitude_deg'],
        lat = df['latitude_deg'],
        text = df['text'],
        mode = 'markers',
        marker = dict(
            size = 7,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Reds',
            cmin = df['departure'].max(),
            color = df['departure'],
            cmax = 0,
            colorbar_title="flights<br>1/2/1401-14/2/1401"
        )))


fig.update_layout(
        title = 'Most trafficked Iran airports',
        geo = dict(
            scope = 'asia',
            resolution = 50,
            lonaxis_range= [43, 64],
            lataxis_range= [24, 42],
            landcolor = 'rgb(217, 217, 217)',
        )
    )
    
fig.show()